In [10]:
import pandas as pd
from rdkit import Chem

csv_path = r'C:\work\DrugDiscovery\main_git\XAI_Chem\data\init_data\pKa_Prediction_Starting data_2024.01.25.csv'

df = pd.read_csv(csv_path, index_col=0)

SMILES_to_pKa = {}
SMILES_to_logP = {}
for index, row in df.iterrows():
    SMILES = row['Smiles']
    SMILES_logP = row['Amides for LogP']
    if pd.isnull(SMILES):
        continue
   
    canon_SMILES = Chem.CanonSmiles(SMILES)
    canon_logP_SMILES = Chem.CanonSmiles(SMILES_logP)

    if canon_SMILES not in SMILES_to_pKa: SMILES_to_pKa[canon_SMILES] = [row['pKa']]
    else: SMILES_to_pKa[canon_SMILES].append(row['pKa']) 

    if canon_logP_SMILES not in SMILES_to_logP: SMILES_to_logP[canon_logP_SMILES] = [row['LogP']]
    else: SMILES_to_logP[canon_logP_SMILES].append(row['LogP']) 

for canon_smi, pKas in SMILES_to_pKa.items():
    if len(pKas) > 1:
        print(canon_smi, pKas)

for canon_smi, LogPs in SMILES_to_logP.items():
    if len(LogPs) > 1:
        print(canon_smi, LogPs)


O=C(O)C1CCC1 [4.5, 4.5]
NC1CCC(F)(F)CC1 [9.45, 9.45]
FC(F)(F)C1CCCN1 [6.06, 6.06]
C1CCNC1 [11.29, 11.21, 11.31]
FC(F)(F)C1CCCCN1 [6.41, 6.41]
C1CCNCC1 [11.12, 11.12]
C1CNC1 [11.13, 11.29]
NCCCC(F)F [10.04, 10.01]
O=C(Nc1ccccc1)C1CCC1 [2.26, 2.26]
O=C(NC1CCC(F)(F)CC1)c1ccccc1 [2.31, 2.32]
O=C(c1ccccc1)N1CCCC1C(F)(F)F [2.12, 2.12]
O=C(c1ccccc1)N1CCCC1 [1.19, 1.19, 1.19]
O=C(c1ccccc1)N1CCCCC1C(F)(F)F [2.54, 2.54]
O=C(c1ccccc1)N1CCCCC1 [1.39, 1.39]
O=C(c1ccccc1)N1CCC1 [1.13, 1.13]
O=C(NCCCC(F)F)c1ccccc1 [1.76, 1.76]


In [11]:
for canon_smi, pKas in SMILES_to_pKa.items():
    SMILES_to_pKa[canon_smi] = round(sum(SMILES_to_pKa[canon_smi]) / len(SMILES_to_pKa[canon_smi]), 2)

for canon_smi, LogPs in SMILES_to_logP.items():
    SMILES_to_logP[canon_smi] = round(sum(SMILES_to_logP[canon_smi]) / len(SMILES_to_logP[canon_smi]), 2)

In [12]:
index_to_remain = []
added_smiles = []
df_canon = df.copy()
for index, row in df_canon.iterrows():
    SMILES = row['Smiles']
    SMILES_logP = row['Amides for LogP']
    if pd.isnull(SMILES):
        continue
   
    canon_SMILES = Chem.CanonSmiles(SMILES)
    canon_logP_SMILES = Chem.CanonSmiles(SMILES_logP)

    if canon_SMILES not in added_smiles:
        index_to_remain.append(index)
    
    added_smiles.append(canon_SMILES)

    df_canon.at[index, 'Smiles'] = canon_SMILES
    df_canon.at[index, 'Amides for LogP'] = canon_logP_SMILES
    df_canon.at[index, 'pKa'] = SMILES_to_pKa[canon_SMILES]
    df_canon.at[index, 'LogP'] = SMILES_to_logP[canon_logP_SMILES]


In [16]:
df_descreased = df_canon.iloc[index_to_remain]

In [17]:
df_descreased.reset_index(inplace=True)

In [18]:
df_descreased.drop(['index'], inplace=True, axis=1)

C:\Users\38066\AppData\Local\Temp\ipykernel_920\931801678.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_descreased.drop(['index'], inplace=True, axis=1)


In [19]:
df_descreased

,Reference,Smiles,LogP,pKa,Structure,MW,Amine or acid,Bemis-Murcko,Framework,F group,...,"Length of linear path, bonds",Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Atoms in ring,Amides for LogP,Unnamed: 19,F atom fraction,identificator
0,10.1016/j.jfluchem.2023.110215,O=C(O)C1CCC(F)(F)CC1,2.63,4.32,NaN,164.152,Acid,NaN,cyclohexane,gem-CF2,...,0,0,2,0,0,6.0,O=C(Nc1ccccc1)C1CCC(F)(F)CC1,NaN,0.2857142857142857,Carboxylic acid
1,10.1016/j.jfluchem.2023.110215,O=C(O)C1CCCCC1,2.88,4.85,NaN,128.171,Acid,NaN,cyclohexane,non-F,...,0,0,0,0,0,6.0,O=C(Nc1ccccc1)C1CCCCC1,NaN,0,Carboxylic acid
2,10.1016/j.jfluchem.2023.110215,O=C(O)[C@H]1C[C@@H]2C[C@@H]2C1,2.87,4.35,NaN,126.155,Acid,NaN,bicyclo[3.1.0]hexane,non-F,...,0,0,0,0,0,6.0,O=C(Nc1ccccc1)[C@H]1C[C@@H]2C[C@@H]2C1,NaN,0,Carboxylic acid
3,10.1016/j.jfluchem.2023.110215,O=C(O)[C@@H]1C[C@@H]2C[C@@H]2C1,2.92,4.24,NaN,126.155,Acid,NaN,bicyclo[3.1.0]hexane,non-F,...,0,0,0,0,0,6.0,O=C(Nc1ccccc1)[C@@H]1C[C@@H]2C[C@@H]2C1,NaN,0,Carboxylic acid
4,10.1016/j.jfluchem.2023.110215,O=C(O)[C@H]1C[C@@H]2[C@H](C1)C2(F)F,2.37,3.80,NaN,162.136,Acid,NaN,bicyclo[3.1.0]hexane,gem-CF2,...,0,0,2,0,0,6.0,O=C(Nc1ccccc1)[C@H]1C[C@@H]2[C@H](C1)C2(F)F,NaN,0.2857142857142857,Carboxylic acid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,10.1021/acs.joc.0c01848,N[C@@H]1C[C@H]1CF,1.41,NaN,NaN,89.113,Amine,NaN,cyclopropane,CH2F,...,0,1,1,0,0,3.0,O=C(N[C@@H]1C[C@H]1CF)c1ccccc1,NaN,0.25,Primary amine
170,10.1021/acs.joc.0c01848,N[C@@H]1C[C@@H]1CF,1.12,NaN,NaN,89.113,Amine,NaN,cyclopropane,CH2F,...,0,1,1,0,0,3.0,O=C(N[C@@H]1C[C@@H]1CF)c1ccccc1,NaN,0.25,Primary amine
171,10.1002/ejoc.202200863,NC1CC12CC2,2.08,9.41,NaN,83.134,Amine,NaN,spiro[2.2]pentane,non-F,...,0,0,0,0,0,5.0,O=C(NC1CC12CC2)c1ccccc1,NaN,0,Primary amine
172,10.1002/ejoc.202200863,NC1C(F)(F)C12CC2,2.14,NaN,NaN,119.115,Amine,NaN,spiro[2.2]pentane,gem-CF2,...,1,1,0,0,0,5.0,O=C(NC1C(F)(F)C12CC2)c1ccccc1,NaN,0.4,Primary amine


In [20]:
df_descreased.to_csv(r'C:\work\DrugDiscovery\main_git\XAI_Chem\data\init_data\pKa_Prediction_Starting data_2024.05.07.csv')